# Geographical Aggregation (Tourism)

> Geographical Hierarchical Forecasting on Australian Tourism Data

In many applications, a set of time series is hierarchically organized. Examples include the presence of geographic levels, products, or categories that define different types of aggregations. In such scenarios, forecasters are often required to provide predictions for all disaggregate and aggregate series. A natural desire is for those predictions to be **"coherent"**, that is, for the bottom series to add up precisely to the forecasts of the aggregated series.

In this notebook we present an example on how to use `HierarchicalForecast` to produce coherent forecasts between geographical levels. We will use the classic Australian Domestic Tourism (`Tourism`) dataset, which contains monthly time series of the number of visitors to each state of Australia.

We will first load the Tourism data and produce base forecasts using a diverse set of models, including `AutoETS` from `StatsForecast`, and machine learning models like `lightgbm` and `HistGradientBoostingRegressor` using `MLForecast`, as well as neural network models like `MLP` and `NBEATS` from `NeuralForecast`. We will then reconcile these base forecasts with several reconciliation algorithms from `HierarchicalForecast`. 

Finally, we show the performance is comparable with the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html) which uses the R package [fable](https://github.com/tidyverts/fable).

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast mlforecast datasetsforecast lightgbm sklearn neuralforecast

## 1. Load and Process Data

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book.

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df["ds"], freq='Q').to_timestamp()
Y_df.head()

,Country,Region,State,Purpose,ds,y
0,Australia,Adelaide,South Australia,Business,1998-01-01,135.077690
1,Australia,Adelaide,South Australia,Business,1998-04-01,109.987316
2,Australia,Adelaide,South Australia,Business,1998-07-01,166.034687
3,Australia,Adelaide,South Australia,Business,1998-10-01,127.160464
4,Australia,Adelaide,South Australia,Business,1999-01-01,137.448533


The dataset can be grouped in the following non-strictly hierarchical structure.

In [ ]:
spec = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
from hierarchicalforecast.utils import aggregate

In [ ]:
%%capture
Y_df, S_df, tags = aggregate(Y_df, spec)

In [ ]:
Y_df.head()

,unique_id,ds,y
0,Australia,1998-01-01,23182.197269
1,Australia,1998-04-01,20323.380067
2,Australia,1998-07-01,19826.640511
3,Australia,1998-10-01,20830.129891
4,Australia,1999-01-01,22087.353380


In [ ]:
S_df.iloc[:5, :5]

,unique_id,Australia/ACT/Canberra/Business,Australia/ACT/Canberra/Holiday,Australia/ACT/Canberra/Other,Australia/ACT/Canberra/Visiting
0,Australia,1.0,1.0,1.0,1.0
1,Australia/ACT,1.0,1.0,1.0,1.0
2,Australia/New South Wales,0.0,0.0,0.0,0.0
3,Australia/Northern Territory,0.0,0.0,0.0,0.0
4,Australia/Queensland,0.0,0.0,0.0,0.0


In [ ]:
tags['Country/Purpose']

array(['Australia/Business', 'Australia/Holiday', 'Australia/Other',
       'Australia/Visiting'], dtype=object)

### Split Train/Test sets

We use the final two years (8 quarters) as test set.

In [ ]:
Y_test_df = Y_df.groupby('unique_id', as_index=False).tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_train_df.groupby('unique_id').size()

unique_id
Australia                                                72
Australia/ACT                                            72
Australia/ACT/Business                                   72
Australia/ACT/Canberra                                   72
Australia/ACT/Canberra/Business                          72
                                                         ..
Australia/Western Australia/Experience Perth/Other       72
Australia/Western Australia/Experience Perth/Visiting    72
Australia/Western Australia/Holiday                      72
Australia/Western Australia/Other                        72
Australia/Western Australia/Visiting                     72
Length: 425, dtype: int64

## 2. Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `ETS` model. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [ ]:
%%capture
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], 
                     freq='QS', n_jobs=-1)
Y_hat_df_stats = fcst.forecast(df=Y_train_df, h=8, fitted=True)
Y_fitted_df_stats = fcst.forecast_fitted_values()

### 2.1 Computing MLForecast models

In [ ]:
%%capture
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingRegressor
from mlforecast.lag_transforms import ExpandingMean, RollingMean, ExpandingStd
from mlforecast.target_transforms import Differences
from mlforecast import MLForecast

In [ ]:
%%capture
mlf = MLForecast(
    models = {
        'lgbm': lgb.LGBMRegressor(verbosity=-1),
        'gbm':HistGradientBoostingRegressor()
    }, 
    freq='QS',
    target_transforms=[Differences([1, 4])],
    lags=[1, 2, 3, 4, 5, 6, 7, 8, 12],
    lag_transforms={  
        1: [ExpandingMean(), RollingMean(window_size=4)],
        4: [ExpandingMean(), RollingMean(window_size=2), RollingMean(window_size=4)],
        8: [RollingMean(window_size=4)]
    },
    date_features=['quarter', 'year']
)
mlf.fit(Y_train_df, fitted=True)
Y_hat_df_ml = mlf.predict(new_df=Y_train_df, h=8)
Y_fitted_df_ml = mlf.forecast_fitted_values()

### 2.2 Computing Neuralforecast models

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP
from neuralforecast.losses.pytorch import MAE

In [ ]:
nf = NeuralForecast(
    models=[
        NBEATS(
            h=4,
            input_size=16,
            mlp_units=[[256, 256], [256, 256], [256, 256]],
            learning_rate=1e-3,
            loss=MAE(),
            random_seed=42
        ),
        MLP(
            h=4,
            input_size=16,
            num_layers=2,
            hidden_size=64,
            max_steps=500,
            learning_rate=1e-3,
            loss=MAE(),
            random_seed=42
        )
    ],
    freq='QS'
)
nf.fit(df=Y_train_df, val_size=4)
Y_hat_df_nf = nf.predict()
Y_fitted_df_nf = nf.predict_insample(step_size=4)

We need to combine the predicted values in order to reconcile the forecast for all models at once.

In [ ]:
%%capture
Y_hat_df = Y_hat_df_stats.merge(Y_hat_df_ml, on=['unique_id', 'ds']).merge(Y_hat_df_nf, on=['unique_id', 'ds'])
Y_fitted_df = Y_fitted_df_stats.merge(Y_fitted_df_ml.drop(columns=['y']), on=['unique_id', 'ds']).merge(Y_fitted_df_nf.drop(columns=['cutoff', 'y']), on=['unique_id', 'ds'], how="left")

## 3. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df.head()

,unique_id,ds,AutoETS,lgbm,gbm,NBEATS,MLP,AutoETS/BottomUp,lgbm/BottomUp,gbm/BottomUp,...,AutoETS/MinTrace_method-mint_shrink,lgbm/MinTrace_method-mint_shrink,gbm/MinTrace_method-mint_shrink,NBEATS/MinTrace_method-mint_shrink,MLP/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols,lgbm/MinTrace_method-ols,gbm/MinTrace_method-ols,NBEATS/MinTrace_method-ols,MLP/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,26523.543195,26718.953482,25906.072266,25596.566406,24381.672901,26945.553515,26760.046498,...,25496.905475,26888.836388,27010.181267,25459.239804,25247.044028,25894.419896,26641.438835,26806.086628,25711.648641,25496.079269
1,Australia,2016-04-01,24458.490282,24900.590405,24975.727629,24768.101562,23770.330078,22903.194016,26028.446233,25827.371356,...,23985.286631,25454.343283,25638.934958,24394.554295,23432.011005,24357.231461,25109.570985,25194.734635,24583.133324,23680.009340
2,Australia,2016-07-01,23974.055984,24139.018873,24453.575388,23682.156250,24544.498047,22411.401317,25345.135452,25210.244053,...,23507.565005,25027.954724,25249.062398,23378.419946,24427.526652,23865.928094,24451.714310,24720.208044,23575.178198,24530.602833
3,Australia,2016-10-01,24563.454495,26281.353485,25888.214226,25000.509766,26054.105469,23127.009693,26969.492032,26780.203246,...,24175.081575,26706.094555,26766.761480,24701.773136,25829.722533,24470.780870,26416.665730,26166.386453,24824.038550,26005.932850
4,Australia/ACT,2016-01-01,553.037571,611.947070,599.386111,554.268188,582.645020,515.900790,621.156907,615.741689,...,556.649119,621.441393,607.954464,554.555923,582.762697,574.658651,591.749093,594.207054,592.450086,599.256119


## 4. Evaluation 

The `HierarchicalForecast` package includes an `evaluate` function to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mase
from functools import partial

We'll be cleaning the columns names and the following function will help us do it in a more structured way:

In [ ]:
def rename_evaluation_columns(
        evaluation_df: pd.DataFrame,
        base_model_name: str = 'AutoETS',
        other_model_names: list = ['AutoETS', 'lgbm', 'knn', 'gbm', 'NBEATS', 'MLP'],
    ) -> pd.DataFrame:

    cleaned_column_mapping = {}

    def _clean_recon_method(raw_name: str) -> str:
        if raw_name.startswith('MinTrace_method-'):
            method_part = raw_name.replace('MinTrace_method-', '')
            return f"MinTrace({method_part})"
        else:
            return raw_name.replace('-', ' ')

    for col in evaluation_df.columns:
        if col in ['level', 'metric']:
            cleaned_column_mapping[col] = col
        elif col == base_model_name:
            cleaned_column_mapping[col] = 'Base'
        elif col.startswith(f'{base_model_name}/'): 
            recon_method_raw = col.split('/', 1)[1]
            cleaned_column_mapping[col] = _clean_recon_method(recon_method_raw)
        else: 
            is_other_base_model = False
            for model in other_model_names:
                if col == model:
                    cleaned_column_mapping[col] = model
                    is_other_base_model = True
                    break
            if is_other_base_model:
                continue
            
            parts = col.split('/', 1)
            if len(parts) == 2:
                model_name = parts[0]
                recon_method_raw = parts[1]
                
                recon_method_clean = _clean_recon_method(recon_method_raw)
                
                new_name = f"{model_name} {recon_method_clean}"
                cleaned_column_mapping[col] = new_name
            else:
                cleaned_column_mapping[col] = col
    
    return evaluation_df.rename(columns=cleaned_column_mapping)

In [ ]:
eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']

df = Y_rec_df.merge(Y_test_df, on=['unique_id', 'ds'])
evaluation = evaluate(df = df,
                      tags = eval_tags,
                      train_df = Y_train_df,
                      metrics = [rmse,
                                 partial(mase, seasonality=4)])

evaluation = rename_evaluation_columns(evaluation)
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

### RMSE

The following table shows the performance measured using RMSE across levels for each reconciliation method.

In [ ]:
evaluation.query('metric == "rmse"')

,level,metric,Base,lgbm,gbm,NBEATS,MLP,BottomUp,lgbm BottomUp,gbm BottomUp,...,MinTrace(mint_shrink),lgbm MinTrace(mint_shrink),gbm MinTrace(mint_shrink),NBEATS MinTrace(mint_shrink),MLP MinTrace(mint_shrink),MinTrace(ols),lgbm MinTrace(ols),gbm MinTrace(ols),NBEATS MinTrace(ols),MLP MinTrace(ols)
0,Total,rmse,963.09,318.13,435.97,848.03,634.04,2271.75,1099.85,950.53,...,1287.58,749.64,901.35,1098.28,873.19,1027.21,433.82,538.81,960.51,698.94
2,Purpose,rmse,348.67,316.69,343.79,398.90,383.82,602.07,380.63,349.54,...,381.13,288.02,330.55,394.29,391.13,340.26,248.80,270.76,363.95,363.51
4,State,rmse,202.85,201.50,181.78,196.41,173.65,315.61,194.73,186.75,...,211.69,186.88,187.06,198.18,176.18,184.65,187.62,169.36,174.88,165.81
6,Regions,rmse,41.75,40.09,42.89,42.07,37.82,45.11,40.66,39.69,...,36.68,40.02,40.96,37.37,36.79,37.65,38.89,40.22,37.91,36.77
8,Bottom,rmse,16.73,18.19,18.02,18.35,16.44,16.73,18.19,18.02,...,15.23,17.97,18.19,16.17,16.07,15.85,18.22,18.79,16.44,16.12
10,Overall,rmse,31.14,29.96,30.54,32.55,29.08,40.00,32.57,31.39,...,30.34,30.28,31.46,30.58,29.34,29.37,29.07,29.89,29.75,28.43


### MASE


The following table shows the performance measured using MASE across levels for each reconciliation method. Focusing only in AutoETS model

In [ ]:
evaluation.query('metric == "mase"')[['level', 'metric', 'Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']]

,level,metric,Base,BottomUp,MinTrace(mint_shrink),MinTrace(ols)
1,Total,mase,0.78,2.38,1.19,0.84
3,Purpose,mase,0.78,1.72,0.90,0.74
5,State,mase,0.98,1.52,0.98,0.84
7,Regions,mase,0.96,1.01,0.85,0.87
9,Bottom,mase,0.88,0.88,0.84,0.91
11,Overall,mase,0.90,0.93,0.84,0.90


In [ ]:
evaluation.query('metric == "mase"')[['level', 'metric', 'Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']]

,level,metric,Base,BottomUp,MinTrace(mint_shrink),MinTrace(ols)
1,Total,mase,0.78,2.38,1.19,0.84
3,Purpose,mase,0.78,1.72,0.90,0.74
5,State,mase,0.98,1.52,0.98,0.84
7,Regions,mase,0.96,1.01,0.85,0.87
9,Bottom,mase,0.88,0.88,0.84,0.91
11,Overall,mase,0.90,0.93,0.84,0.90


### Comparison fable

Observe that we can recover the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html). The original results were calculated using the R package [fable](https://github.com/tidyverts/fable).

![Fable's reconciliation results](./imgs/AustralianDomesticTourism-results-fable.png)

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Rob Hyndman, Alan Lee, Earo Wang, Shanika Wickramasuriya, and Maintainer Earo Wang (2021). "hts: Hierarchical and Grouped Time Series". URL https://CRAN.R-project.org/package=hts. R package version 0.3.1.](https://cran.r-project.org/web/packages/hts/index.html)
- [Mitchell O’Hara-Wild, Rob Hyndman, Earo Wang, Gabriel Caceres, Tim-Gunnar Hensel, and Timothy Hyndman (2021). "fable: Forecasting Models for Tidy Time Series". URL https://CRAN.R-project.org/package=fable. R package version 6.0.2.](https://CRAN.R-project.org/package=fable)